In [ ]:
#Cropping TIFF images of size 1024x1024 to four 512x512 (png) tiles
import os
from contextlib import contextmanager
import rasterio
from rasterio import Affine
from rasterio.enums import Resampling
import glob
from pathlib import Path
from itertools import product
import rasterio as rio
from rasterio import windows
            
output = '//Users/rohullahnajibi/desktop/Berkeley_Lab/RTU_Bounding_Box/cropped_B&W_masks/'
input_file  = '/Users/rohullahnajibi/desktop/Berkeley_Lab/RTU_Bounding_Box/Black_&_White_Labels/'
if not os.path.exists(output):
    os.mkdir(output)
def get_tiles(ds, width, height):
    ncols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, ncols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=ncols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform
for filename in glob.glob(os.path.join(input_file, '*.png')):
    with rio.open(filename) as inds:
        tile_width, tile_height = 512, 512
        i, j = -1, 0
        meta = inds.meta.copy()
        for window, transform in get_tiles(inds, tile_width, tile_height):
            meta['transform'] = transform
            meta['width'], meta['height'] = window.width, window.height
            if j%5 == 0:
                j = 0
                i += 1
            string = output + Path(filename).stem + '_' + str(i) + '-' + str(j) + '.png'
            j += 1
            with rio.open(string, 'w', **meta) as outds:
                outds.write(inds.read(window=window))
                
#This code gives the extension .png to every image, which is good for the masks but not the images. 
#To rename .png to .jpg for the images, here is my code:

#converting images from png to jpg
import os
in_put = '/Users/rohullahnajibi/desktop/berkeley_lab/RTU_Bounding_Box/raw_cropped_img'
for filename in os.listdir(in_put):
    if filename.endswith('.png'):
        os.rename(os.path.join(in_put, filename), os.path.join(in_put, filename.replace('png', 'jpg')))
